### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
# cell for work
reviews.rating.max()

10

In [3]:
# cell for work
movies.shape[0]

34508

In [18]:
# more work
# finding number of genres
genres = []

for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

genres = set(genres)
print(len(genres))

28


In [5]:
# and still more
reviews.user_id.nunique()

60446

In [6]:
# and even more - open additional cells as necessary

In [7]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 34496,
'The number of ratings in the dataset': 816337,
'The number of different genres':28,
'The number of unique users in the dataset':60431,
'The number missing ratings in the reviews dataset':0,
'The average rating given across all ratings':7,
'The minimum rating given across all ratings':0,
'The maximum rating given across all ratings':10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [12]:
movies.head()

movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  
0  Documentary|Short  
1  Documentary|Short  
2  Documentary|Short  
3                NaN  
4       Short|Horror

In [19]:
# cell for work
# pull date if it exists
create_date = lambda val: val[-5 : -1] if val[-1] == ')' else np.nan

# apply the function
movies['date'] = movies['movie'].apply(create_date)

# return century of a movie as dummy column
def movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
    
for yr in ['18', '19', '20']:
    movies[str(yr) + "00s"] = movies['date'].apply(movie_year)
    
movies.head()

movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  date  1800s  1900s  2000s  
0  Documentary|Short  1894      1      0      0  
1  Documentary|Short  1895      1      0      0  
2  Documentary|Short  1896      1      0      0  
3                NaN  1895      1      0      0  
4       Short|Horror  1896      1      0      0

In [22]:
# Function to split the genres
def split_genre(val):
    try:
        if val.find(gene) > -1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply the function
for gene in genres:
    movies[gene] = movies['genre'].apply(split_genre)
    
movies.head()

movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  date  1800s  1900s  2000s  Documentary  Comedy  \
0  Documentary|Short  1894      1      0      0            1       0   
1  Documentary|Short  1895      1      0      0            1       0   
2  Documentary|Short  1896      1      0      0            1       0   
3                NaN  1895      1      0      0            0       0   
4       Short|Horror  1896      1      0      0            0       0   

   Thriller   ...     Game-Show  Talk-Show  Music  Film-Noir  Mystery  \
0         0   ...             0          0      0          0        0   
1         0   ...             0          0      0          0        0   
2         0   ...             0          0      0          0        0   
3         0   ...             0          0      0          0        0   
4         0   ...             0          0      0          0        0   

   History  Action  Short  Sci-Fi  Musical  
0        0       0      1       0        0  
1        0       0      1       0        0  
2        0       0      1       0        0  
3        0       0      0       0        0  
4        0       0      1       0        0  

[5 rows x 35 columns]

In [24]:
# Function
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [25]:
# and a few more below, which you can use as necessary
reviews.to_csv('./reviews_final.csv')
movies.to_csv('./movies_final.csv')

In [11]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year